## Esquema del flujo de trabajo en el notebook

1. **Importación de librerías**
   - Se importan todas las librerías necesarias para análisis de datos, visualización y modelado.

2. **Preprocesamiento de los datos**
   - **Carga de datos de calidad:**  
     - Se importan varias hojas del archivo Excel de calidad usando `pd.read_excel()`.
     - Se eliminan columnas irrelevantes y filas innecesarias (encabezados extra, pies de página).
     - Se normalizan los nombres de las columnas.
     - Se limpian y depuran los DataFrames individuales.
   - **Unificación de datos de calidad:**  
     - Se concatenan los DataFrames de las diferentes hojas en uno solo (`df_Calidad`).
   - **Procesamiento de las notas de catación:**  
     - Se limpian y convierten las notas de catación en listas.
     - Se asegura que todas las celdas sean listas, incluso si hay valores nulos.
     - Se aplica one-hot encoding para convertir cada nota en una variable binaria.
   - **Visualización y verificación:**  
     - Se muestran las primeras y últimas filas del DataFrame para verificar la limpieza.

3. **Preprocesamiento de los datos de tostión**
   - **Carga de datos de tostión:**  
     - Se importan varias hojas del archivo Excel de tostión.
     - Se eliminan columnas y filas irrelevantes.
     - Se normalizan los nombres de las columnas.
   - **Unificación de datos de tostión:**  
     - Se concatenan los DataFrames de las diferentes hojas en uno solo (`df_Tostion`).
     - Se verifica la consistencia y los tipos de datos.

4. **Unión de los datos de calidad y tostión**
   - Se realiza un merge entre `df_Calidad` y `df_Tostion` usando la columna `LOTE` como clave.
   - Se obtiene un único DataFrame (`df_full`) listo para análisis y modelado.

5. **Análisis y modelado**
   - **Selección de variables predictoras (X) y variable objetivo (Y).**
   - **Entrenamiento de modelos de regresión** (por ejemplo, Linear Regression, Random Forest).
   - **Evaluación y comparación de modelos** usando métricas como RMSE, MAE, R².
   - **Explicabilidad**: análisis de importancia de variables, SHAP, LIME, etc.
   - **Reporte de resultados** y visualizaciones finales.

---

Este esquema refleja el orden real y lógico de los procedimientos realizados en el notebook, comenzando por el preprocesamiento y asegurando la calidad de los datos antes de cualquier análisis o modelado. 


In [127]:
#Primero importamos todas las librerias que vamos a utilizar
import os 
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Importamos las librerias de sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report



### 1. Procesamiento de datos

En esta etapa preparamos y examinamos el conjunto de datos para garantizar su calidad antes de cualquier análisis o modelado. El flujo seguido es el siguiente:

1. **Carga del conjunto de datos**  
   - Utilizamos `pd.read_excel()` para importar varias hojas relevantes del archivo Excel, especificando los parámetros `sheet_name` y `skiprows` para ubicar correctamente los encabezados y omitir filas innecesarias.
   - Cada hoja se carga como un DataFrame independiente para su limpieza inicial.

2. **Limpieza y depuración de columnas y filas**  
   - Eliminamos columnas irrelevantes o con valores constantes que no aportan información al análisis.
   - Eliminamos filas de encabezado adicionales, filas vacías o de pie de página que no contienen datos útiles.
   - Normalizamos los nombres de las columnas, quitando espacios al inicio y final.

3. **Unificación de datos**  
   - Unimos los DataFrames de las diferentes hojas en uno solo usando `pd.concat()`, asegurando que toda la información relevante esté en un único DataFrame.

4. **Procesamiento de las notas de catación**  
   - Las notas de catación, originalmente en una sola columna como texto separado por comas, se procesan para garantizar uniformidad:
     - Se eliminan espacios redundantes y se divide el texto en listas de notas.
     - Se asegura que todas las celdas sean listas, incluso si hay valores nulos.
   - Posteriormente, se aplica un proceso de one-hot encoding:
     - Cada nota de catación se convierte en una columna binaria (0/1) usando `str.get_dummies()`, indicando la presencia o ausencia de esa característica en cada muestra.
     - Esto permite que los modelos interpreten correctamente las variables categóricas y facilita la interpretación de la importancia de cada nota.

5. **Visualización y verificación**  
   - Se muestran las primeras y últimas filas del DataFrame resultante con `display(df.head())` y `display(df.tail())` para verificar que la limpieza y transformación se realizaron correctamente.

Con este proceso, aseguramos que el DataFrame esté limpio, estructurado y listo para el análisis y modelado, con variables categóricas correctamente codificadas y sin información redundante o errónea.


In [128]:
#Primero Cargamos los datos almacenados en los archivos .xlsx
#Las primeras 5 filas de los archivos son encabezados, por lo cual las saltamos

dict_Calidad = pd.read_excel('CC_FT_17_Calidad.xlsx', 
                             sheet_name=['CONTROL CALIDAD CAFE TRILLADO J','Sheet2'],
                             skiprows=5)

#Accedemos a las hojas específicas del diccionario
#Quitamos las columnas que no son necesarias para el análisis, por ejemplo verificación física del café tostado tiene 
#Unicamente un valor y no aporta información relevante para el análisis

df_Calidad1 = dict_Calidad['CONTROL CALIDAD CAFE TRILLADO J']

#Las columnas sin nombre en este caso solo tienen un valor: 'C', por lo cual procedemos a eliminarlas.
df_Calidad1 = df_Calidad1.loc[:, ~df_Calidad1.columns.str.startswith('Unnamed')]

#quita espacios al principio y al final de cada nombre
df_Calidad1.columns = df_Calidad1.columns.str.strip()

df_Calidad1 = df_Calidad1.drop(columns=['FECHA','VERIFICACIÓN FISICA CAFÉ TOSTADO', 'LIBERACIÓN DE LOTE', 'RESPONSABLE'])

#Por ultimo eliminamos las filas 0 y 1 que no aportan información relevante y las ultimas filas.

df_Calidad1 = df_Calidad1.drop(index=[0,1])
df_Calidad1 = df_Calidad1.iloc[:-17]


# Realizamos una visualización rápida de los datos en cada hoja

display(df_Calidad1.head())
display(df_Calidad1.tail())


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-190722,Madre Laura,765.00,10.9,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,09-190722,Tabi Natural,204.00,10.2,14,"Frutas maduras, nibs de cacao, acidez brillant...",85
4,10-190722,Don Mario,165.00,10.7,14,"Panela, durazno, miel, acidez brillante citric...",84.5
5,07-19-07-22,Don Felix,0.45,10.5,14,"Moras maduras, chocolate negro, acidez media c...",84.5
6,01-291022,Madre Laura,105.00,10.7,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
73,22-170624,El Ocaso - Caturron,48.0,10.3,15,"Chocolate dulce, nuez moscada, acidez jugosa, ...",87
74,01-030724,Madre Laura,250.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
75,23-150724,Esteban Robledo,60.0,10.6,14,"Panela, almendras, acidez citrica jugosa como ...",85
76,02-150724,Madre Laura Natural,175.0,10.8,14,"Cacao, frutos rojos, cuerpo cremoso, acidez br...",85
77,17-270724,Familia Bedoya Castaño,35.0,10.8,14,"Melao de panela, arandanos, cuerpo cremoso, ac...",87


In [129]:
#Ahora hacemos un tratamiento similar con la segunda hoja del archivo Excel
df_Calidad2 = dict_Calidad['Sheet2']

df_Calidad2 = df_Calidad2.loc[:, ~df_Calidad2.columns.str.startswith('Unnamed')]
df_Calidad2.columns = df_Calidad2.columns.str.strip()
df_Calidad2 = df_Calidad2.drop(columns=['FECHA','VERIFICACIÓN FISICA CAFÉ TOSTADO', 'LIBERACIÓN DE LOTE', 'RESPONSABLE'])
df_Calidad2 = df_Calidad2.drop(index=[0,1])
df_Calidad2 = df_Calidad2.iloc[:-12]

display(df_Calidad2.head())
display(df_Calidad2.tail())

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-300822,Madre Laura,32.0,"10,5",14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,01-131022,Madre Laura,79.8,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
4,01-181022,Madre Laura,38.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
5,05-181022,Doña Dolly,43.0,10,14,"Chocolate dulce,fresas,miel,cuerpo cremoso,aci...",85
6,01-291022,Madre Laura,272.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
29,01-271023,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
30,01-100124,Madre Laura,20.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
31,01-020424,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
32,01-200624,Madre Laura,14.0,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
33,01-180724,Madre Laura,20.0,10.2,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


In [130]:
#Ahora unimos los dos dataframes en uno solo por filas

df_Calidad = pd.concat([df_Calidad1, df_Calidad2], axis=0, ignore_index=True)

display(df_Calidad.head())
display(df_Calidad.tail())

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
0,01-190722,Madre Laura,765.00,10.9,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
1,09-190722,Tabi Natural,204.00,10.2,14,"Frutas maduras, nibs de cacao, acidez brillant...",85
2,10-190722,Don Mario,165.00,10.7,14,"Panela, durazno, miel, acidez brillante citric...",84.5
3,07-19-07-22,Don Felix,0.45,10.5,14,"Moras maduras, chocolate negro, acidez media c...",84.5
4,01-291022,Madre Laura,105.00,10.7,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
103,01-271023,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
104,01-100124,Madre Laura,20.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
105,01-020424,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
106,01-200624,Madre Laura,14.0,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
107,01-180724,Madre Laura,20.0,10.2,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


Las notas de cata del café constituyen variables categóricas muy relevantes para el modelo. Para incorporarlas sin introducir un orden arbitrario, aplicamos one-hot encoding: creamos una columna por cada nota de cata (por ejemplo, “Sabor afrutado”, “Cuerpo ligero”, “Aroma floral”, etc.) y asignamos un 1 si el café presenta dicha nota, o un 0 en caso contrario.

Este enfoque:

- Mantiene intacta la información cualitativa de cada nota sin reducirla a un único valor numérico.
- Facilita que los algoritmos de regresión interpreten correctamente la presencia o ausencia de cada característica sensorial.
- Permite evaluar la importancia individual de cada nota mediante técnicas de explicabilidad (Feature Importance, SHAP, LIME).

De este modo, garantizamos que el modelo reciba entradas limpias y libres de sesgos de orden, y podamos extraer conclusiones precisas sobre qué atributos de cata influyen más en la calidad final del café.


In [131]:
#Primero modificamos los datos de la columna 'NOTAS DE CATACIÓN' para convertirlos en una lista de notas
# El proceso a continuacion nos garantiza la uniformidad de los datos, pues garantiza que cada nota este en un formato uniforme.
# Limpia espacios redundantes y divide en lista
df_Calidad['NOTAS DE CATACIÓN'] = (
    df_Calidad['NOTAS DE CATACIÓN']
      .fillna('')                           # en caso de NaN
      .str.replace(r'\s*,\s*', ',', regex=True)
      .str.split(',')                       # ahora cada fila es lista de notas
)

display(df_Calidad.head(10))

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
0,01-190722,Madre Laura,765.00,10.9,14,"[Chocolate negro, toque frutal, cuerpo medio, ...",84
1,09-190722,Tabi Natural,204.00,10.2,14,"[Frutas maduras, nibs de cacao, acidez brillan...",85
2,10-190722,Don Mario,165.00,10.7,14,"[Panela, durazno, miel, acidez brillante citri...",84.5
3,07-19-07-22,Don Felix,0.45,10.5,14,"[Moras maduras, chocolate negro, acidez media ...",84.5
4,01-291022,Madre Laura,105.00,10.7,14,"[Chocolate negro, toque frutal, cuerpo medio, ...",84
5,01-291022,Madre Laura,105.00,10.7,14,"[Chocolate negro, toque frutal, cuerpo medio, ...",84
6,01-101122,Madre Laura,210.00,10.7,14,"[Chocolate negro, toque frutal, cuerpo medio, ...",84
7,08-241122,Gesha Villabernarda,12.00,10.3,14,"[Jazmin, mandarina, acidez jugosa, cuerpo sedo...",86
8,02-241122,Madre Laura Natural,19.40,10.5,14,"[Cacao, frutos rojos, cuerpo cremoso, acidez b...",85
9,05-241122,Doña Dolly,35.00,10.8,14,"[Panela, fresa, acidez brillante, cuerpo cremo...",85


### ¿Qué hace `str.get_dummies(sep=',')` en el código a continuación?

1. **Lectura de cada celda como texto**  
   Extrae el contenido de la celda donde las notas están separadas por comas.

2. **Separación por comas**  
   Divide esa cadena en fragmentos individuales usando la coma como separador.

3. **Creación de columnas únicas**  
   Por cada fragmento distinto detectado en todo el DataFrame, genera una nueva columna.

4. **Asignación de valores binarios (0/1)**  
   Asigna un `1` en cada fila si el fragmento aparece en la celda original, o un `0` si no.

In [132]:
# Primero vuelve a la versión “unida en un string” para facilitar la creación de dummies:
df_Calidad['_tmp'] = df_Calidad['NOTAS DE CATACIÓN'].apply(lambda lst: ','.join(lst) if isinstance(lst, list) else '')

# Luego crea dummies (cada nota = columna 0/1)
dummies = df_Calidad['_tmp'].str.get_dummies(sep=',')
dummies.columns = [col.strip() for col in dummies.columns] 
display(dummies.head(10))


,Arandanos,Azucar morena,Cacao,Caña de azucar,Cereza,Chocolate dulce,Chocolate negro,Frambuesa,Fresa,Frutas amarillas,...,naranja,nibs de cacao,nueces,nuez moscada,panela,residual a cascara de mandarina,residual frutal a naranja,te de cedron,toque frutal,vainilla
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [133]:
# Junta al df original lo producido anteriormente
# Esto elimina la columna temporal y la original de notas de cata, dejando solo las dummies
# Esto nos permite tener un dataframe con las notas de cata como columnas, donde cada columna 
# representa una nota y su valor es 0 o 1 dependiendo de si la nota esta presente o no en la fila.
df_Calidad = pd.concat([df_Calidad.drop(columns=['_tmp','NOTAS DE CATACIÓN']), dummies], axis=1)
df_Calidad.head(10)

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,PUNTAJE,Arandanos,Azucar morena,Cacao,Caña de azucar,...,naranja,nibs de cacao,nueces,nuez moscada,panela,residual a cascara de mandarina,residual frutal a naranja,te de cedron,toque frutal,vainilla
0,01-190722,Madre Laura,765.00,10.9,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,09-190722,Tabi Natural,204.00,10.2,14,85,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,10-190722,Don Mario,165.00,10.7,14,84.5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,07-19-07-22,Don Felix,0.45,10.5,14,84.5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01-291022,Madre Laura,105.00,10.7,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,01-291022,Madre Laura,105.00,10.7,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,01-101122,Madre Laura,210.00,10.7,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,08-241122,Gesha Villabernarda,12.00,10.3,14,86,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,02-241122,Madre Laura Natural,19.40,10.5,14,85,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,05-241122,Doña Dolly,35.00,10.8,14,85,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Procesamiento de datos de tostión y fusión de tablas

A continuación incorporamos la información de la etapa de tostión:

1. **Carga y preprocesamiento de la hoja de tostión**  
   - Leemos el archivo `CC_FT_18_Tostion.xlsx` con `pd.read_excel`, apuntando a la hoja correspondiente.  
   - Aplicamos limpieza y transformación de columnas:  
     - Eliminación de filas de encabezado o pie de página innecesarios.  
     - Conversión de tipos de dato (fechas, numéricos).  
     - Normalización de nombres de columnas (quita espacios, tildes uniformes).

2. **Unión con el DataFrame de calidad**  
   - Realizamos un _merge_ entre `df_Calidad` y el nuevo DataFrame `df_Tostion` usando la columna común `LOTE`, que identifica de forma única cada lote de café.  
   - Empleamos por ejemplo:  
     ```python
     df_full = pd.merge(
         df_Calidad,
         df_Tostion,
         on='LOTE',
         how='inner'   # conserva solo los lotes presentes en ambas tablas
     )
     ```  
   - Con esta fusión obtenemos un único DataFrame enriquecido, que combina las variables sensoriales de calidad con los parámetros de tostión, listo para el análisis y el entrenamiento de nuestros modelos de regresión.


In [134]:
# Cargamos el archivo de datos de tostion del café
dict_Tostion = pd.read_excel('CC_FT_18_Tostion.xlsx', 
                           sheet_name=['TOSTIÓN JERICÓ L', 'TOSTIÓN JERICÓ'],
                           skiprows=5)

# Accedemos a las hojas específicas del diccionario
df_Tostion1 = dict_Tostion['TOSTIÓN JERICÓ L']
df_Tostion2 = dict_Tostion['TOSTIÓN JERICÓ']

# Visualizamos los datos de tostion
display(df_Tostion1.head())
display(df_Tostion1.tail())


,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
0,22-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,08:01:00,NaN,LFQ
1,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,175°/195°,08:42:00,NaN,LFQ
2,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,07:58:00,NaN,LFQ
3,28-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,08:02:00,NaN,LFQ
4,28-07-22,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,150°/186°,08:10:00,NaN,LFQ


,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
494,13-8-24,01-120424,Jerico,Dos mil,Tradicional,Lavado,54.0,16.666667,45.0,Filtrados,180°/193°,08:58:00,NaN,LFQ
495,13-8-24,08-140524,Jerico,Gesha,Honey,Honey,3.0,16.666667,2.5,Filtrados,175°/191°,08:10:00,NaN,LFQ
496,13-8-24,22-170624,Salento,Caturron,Natural,Natural,3.0,16.666667,2.5,Filtrados,175°/189°,08:01:00,NaN,LFQ
497,13-8-24,21-130224,Planadas,Red Bourbon,Honey,Honey,3.0,16.666667,2.5,Filtrados,170°/190°,08:09:00,NaN,LFQ
498,13-8-24,23-150724,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.5,Filtrados,175°/191°,08:21:00,NaN,LFQ


In [135]:
display(df_Tostion2.head())
display(df_Tostion2.tail())

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
0,2022-10-08 00:00:00,1-190722,Jerico,Colombia,Tradicional,Lavado,3.0,15.000000,2.55,Filtrado,180°-190°,9:58,NaN,AC
1,2022-10-08 00:00:00,1-190722,Jerico,Colombia,Tradicional,Lavado,3.0,15.000000,2.55,Filtrado,180°-181°,8:32,NaN,AC
2,20-09-2022,01-300822,Jerico,Colombia,Tradicional,Lavado,3.0,15.000000,2.55,Filtrado,180°-190°,9:58,NaN,AC
3,13-10-2022,01-131022,Jerico,Colombia,Tradicional,Lavado,93.0,16.666667,77.50,Espresso,180°-195°,11:40,Epoca,AC
4,18-10-2022,01-181022,Jerico,Colombia,Tradicional,Lavado,42.0,14.285714,36.00,Filtrado,180°-192°,10.15,NaN,LFQ


,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
25,27-10-2023,01-271023,Jerico,Colombia,Tradicional,Lavado,15.60,15.705128,13.15,Filtrado,180°-190°,9.36,Jericó,AC
26,2024-01-10 00:00:00,01-100124,Jerico,Colombia,Tradicional,Lavado,15.00,13.333333,13.00,Filtrado,180°-190°,9.32,Jericó,AC
27,2024-04-02 00:00:00,01-020424,Jerico,Colombia,Tradicional,Lavado,14.50,16.551724,12.10,Filtrado,180°-190°,9.36,Jericó,AC
28,2024-06-20 00:00:00,01-200624,Jerico,Colombia,Tradicional,lavado,10.70,13.551402,9.25,Filtrado,180°-190°,9.28,Jericó,AC
29,2024-07-18 00:00:00,01-180724,Jerico,Colombia,Tradicional,lavado,15.65,15.654952,13.20,Filtrado,180°-190°,9.30,Jericó,AC


In [136]:
# Unimos los dos dataframes de tostion en uno solo por filas
df_Tostion = pd.concat([df_Tostion1, df_Tostion2], axis=0, ignore_index=True)
display(df_Tostion.head())
display(df_Tostion.tail())

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
0,22-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,08:01:00,NaN,LFQ
1,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,175°/195°,08:42:00,NaN,LFQ
2,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,07:58:00,NaN,LFQ
3,28-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,08:02:00,NaN,LFQ
4,28-07-22,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,150°/186°,08:10:00,NaN,LFQ


,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
524,27-10-2023,01-271023,Jerico,Colombia,Tradicional,Lavado,15.60,15.705128,13.15,Filtrado,180°-190°,9.36,Jericó,AC
525,2024-01-10 00:00:00,01-100124,Jerico,Colombia,Tradicional,Lavado,15.00,13.333333,13.00,Filtrado,180°-190°,9.32,Jericó,AC
526,2024-04-02 00:00:00,01-020424,Jerico,Colombia,Tradicional,Lavado,14.50,16.551724,12.10,Filtrado,180°-190°,9.36,Jericó,AC
527,2024-06-20 00:00:00,01-200624,Jerico,Colombia,Tradicional,lavado,10.70,13.551402,9.25,Filtrado,180°-190°,9.28,Jericó,AC
528,2024-07-18 00:00:00,01-180724,Jerico,Colombia,Tradicional,lavado,15.65,15.654952,13.20,Filtrado,180°-190°,9.30,Jericó,AC


In [137]:
# Procedemos a asegurar que los datos de tostion son consistentes, en especial el nombre de las columnas

df_Tostion.columns = df_Tostion.columns.str.strip()  # Quita espacios al principio y al final de cada nombre

In [138]:
# Ahora se procesan los datos de tostion:
# Quitamos las columnas que no son necesarias para el análisis, por ejemplo la columna de fecha y observaciones
# que no aportan información relevante para el análisis

df_Tostion = df_Tostion.drop(columns=['Fecha', 'Observaciones','Tostador'])
# Visualizamos los datos de tostion después de la limpieza
display(df_Tostion.head())

#adicionalmente, examinamos los tipos de datos de las columnas para asegurarnos de que son correctos
print(df_Tostion.dtypes)

,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste
0,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,08:01:00
1,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,175°/195°,08:42:00
2,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,07:58:00
3,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,08:02:00
4,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,150°/186°,08:10:00


Lote                        object
Origen                      object
Variedad                    object
Proceso                     object
Beneficio                   object
Peso en Verde              float64
Merma                      float64
Peso en Tostado            float64
Perfil                      object
Temp. De inicio y final     object
Tiempo de tueste            object
dtype: object


### Unión de los datos

Una vez que los datos de calidad y de tostión han sido limpiados y procesados por separado, procedemos a integrarlos en un solo DataFrame para el análisis final y el modelado.

- **Clave de unión:**  
  Utilizamos la columna `LOTE` como identificador único, ya que está presente en ambos conjuntos de datos y permite relacionar correctamente la información sensorial (calidad) con los parámetros de tostión para cada muestra de café.

- **Método de unión:**  
  Empleamos la función `pd.merge()` de pandas para realizar la fusión de los DataFrames:
  - Usamos el parámetro `on='LOTE'` para indicar la columna clave.
  - El parámetro `how='inner'` asegura que solo se conserven los lotes que están presentes en ambos conjuntos de datos, garantizando la consistencia de la información.

- **Resultado:**  
  El DataFrame resultante contiene, para cada lote, tanto las variables sensoriales y categóricas de calidad como los datos técnicos de tostión. Esto permite realizar análisis integrales y entrenar modelos de regresión que consideren todas las variables relevantes.

**Ejemplo de código para la unión:**
```python
df_full = pd.merge(
    df_Calidad,
    df_Tostion,
    on='Lote',
    how='inner'  # Solo lotes presentes en ambos DataFrames
)
display(df_full.head())
```
De este modo, obtenemos un único DataFrame listo para el análisis exploratorio, la selección de variables y el entrenamiento

In [139]:
# Ahora que tenemos los datos de calidad y tostion, podemos proceder a unirlos
# Primero aseguramos que las columnas de unión son las mismas en ambos dataframes

df_full = pd.merge(df_Calidad, df_Tostion, left_on='LOTE', right_on='Lote', how='inner')

display(df_full.head())
df_full.info()

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,PUNTAJE,Arandanos,Azucar morena,Cacao,Caña de azucar,...,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste
0,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,175°/191°,08:01:00
1,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,175°/195°,08:42:00
2,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,175°/190°,07:58:00
3,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,175°/191°,08:02:00
4,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,175°/192°,08:02:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469 entries, 0 to 468
Columns: 113 entries, LOTE to Tiempo de tueste
dtypes: float64(4), int64(96), object(13)
memory usage: 414.2+ KB


Por último, es necesario transformar las columnas 'Temp. De inicio y final' y 'Tiempo de tueste' a un formato numérico para que puedan ser utilizadas correctamente en el análisis y modelado.  
- En la columna 'Temp. De inicio y final', se extraen las temperaturas de inicio y final como valores numéricos separados, creando dos nuevas columnas: 'Temp_inicio' y 'Temp_final'.  
- En la columna 'Tiempo de tueste', se convierte el tiempo registrado en formato de texto (por ejemplo, '08:01:00') a minutos o segundos, generando una nueva columna numérica que representa la duración total del tueste.  
De esta manera, ambas variables quedan listas para ser utilizadas por los modelos de machine learning.

In [148]:
# Procesar columna 'Temp. De inicio y final'
df_full[['Temp_inicio', 'Temp_final']] = df_full['Temp. De inicio y final'].str.extract(r'(\d+)[°º]*/(\d+)[°º]*').astype(float)
# Procesar columna 'Tiempo de tueste' a minutos
# Procesar columna 'Tiempo de tueste' cuando todas las entradas son datetime.time
df_full['Tiempo_tueste_min'] = df_full['Tiempo de tueste'].apply(lambda t: t.hour * 60 + t.minute + t.second / 60)

AttributeError: 'str' object has no attribute 'hour'

In [140]:
# Desplegamos la información del dataframe completo para verificar que la unión se realizó correctamente
df_full.info()
df_full.shape
df_full.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469 entries, 0 to 468
Columns: 113 entries, LOTE to Tiempo de tueste
dtypes: float64(4), int64(96), object(13)
memory usage: 414.2+ KB


['LOTE',
 'DENOMINACIÓN/     MARCA',
 'CANTIDAD',
 '%H',
 'MALLAS',
 'PUNTAJE',
 'Arandanos',
 'Azucar morena',
 'Cacao',
 'Caña de azucar',
 'Cereza',
 'Chocolate dulce',
 'Chocolate negro',
 'Frambuesa',
 'Fresa',
 'Frutas amarillas',
 'Frutas maduras',
 'Jalea',
 'Jazmin',
 'Lavanda',
 'Limonaria',
 'Manzana roja',
 'Maracuya',
 'Melao de panela',
 'Moras maduras',
 'Panela',
 'Rosas',
 'Sandia',
 'Uva',
 'acidez brillante',
 'acidez brillante citrica',
 'acidez brillante citrica como mandarina',
 'acidez brillante y final frutal',
 'acidez brillante y residual frutal a naranja',
 'acidez citrica como la naranja',
 'acidez citrica jugosa como la mandrina',
 'acidez jugosa',
 'acidez jugosa citrica',
 'acidez jugosa citrica y final frutal.',
 'acidez jugosa como tamarindo',
 'acidez malica',
 'acidez malica brillante',
 'acidez media citrica como naranja',
 'almendras',
 'arandanos',
 'balanaceado y final limpio.',
 'caramelo',
 'caramleo',
 'cedro',
 'chocolate al 70%',
 'chocolate 

### 2. Modelo y análisis

En esta sección se abordará el modelado predictivo de la calidad del café utilizando los datos previamente procesados y fusionados. El objetivo es predecir la variable de calidad (score) a partir de las características sensoriales y técnicas de cada lote.

El procedimiento será el siguiente:

1. **Selección de variables**
   - Se identifican las variables predictoras (X) y la variable objetivo (Y) a partir del DataFrame `df_full`.
   - Antes del modelado, se realizará un procedimiento adicional para el procesamiento de los datos cualitativos (categóricos). Se utilizará un **pipeline** de scikit-learn junto con **OneHotEncoder** para transformar automáticamente las variables categóricas en variables numéricas (dummies) dentro del flujo de entrenamiento y validación de los modelos.  
Esto asegura que cualquier columna categórica sea correctamente codificada y que el preprocesamiento sea reproducible y consistente durante todo el proceso de

2. **División de los datos**
   - Se separan los datos en tres conjuntos: entrenamiento, validación y prueba, utilizando `train_test_split`. Esto permite entrenar los modelos, ajustar hiperparámetros y evaluar el desempeño de manera objetiva

3. **Entrenamiento de modelos de regresión**
   - Se entrenarán dos modelos de regresión:
     - **RandomForestRegressor:** Modelo de ensamble basado en árboles de decisión, robusto ante variables categóricas y capaz de capturar relaciones no lineales.
     - **XGBRegressor:** Modelo de boosting basado en árboles, eficiente y con gran capacidad de predicción en datos tabulares.

4. **Evaluación de desempeño**
   - Se evaluarán ambos modelos utilizando métricas como RMSE (Root Mean Squared Error), MAE (Mean Absolute Error) y R² (coeficiente de determinación).
   - Se compararán los resultados para seleccionar el modelo más adecuado.

5. **Interpretación y explicabilidad**
   - Se analizará la importancia de las variables en cada modelo para identificar qué características influyen más en la calidad del café.
   - Se utilizarán dos enfoques principales:
     - **Feature Importance:** Permite visualizar y cuantificar la contribución de cada variable al modelo, especialmente útil en RandomForestRegressor y XGBRegressor.
     - **SHAP (SHapley Additive exPlanations):** Proporciona una explicación más detallada y local/global sobre cómo cada variable afecta las predicciones del modelo, ayudando a interpretar el impacto individual de cada característica en la predicción de la calidad del café.

6. **Visualización de resultados**
   - Se presentarán gráficas comparando las predicciones de los modelos con los valores reales y se mostrarán los principales hallazgos.

Este enfoque permite comparar dos de los modelos más potentes para regresión en datos tabulares y obtener conclusiones sólidas sobre los factores que determinan la calidad del café.

En los datos algunas variables que deberían ser numéricas (por ejemplo: %H o PUNTAJE) están siendo interpretadas como categóricas (tipo 'object'). Por ello, es necesario convertir estas columnas al tipo numérico adecuado antes de continuar con el preprocesamiento y modelado, para asegurar que los algoritmos las traten correctamente y evitar errores

In [141]:
# Cambiamos los tipos de datos de las columnas que deberían ser numéricas
df_full['%H'] = pd.to_numeric(df_full['%H'], errors='coerce')
df_full['PUNTAJE'] = pd.to_numeric(df_full['PUNTAJE'], errors='coerce')
df_full['MALLAS'] = pd.to_numeric(df_full['MALLAS'], errors='coerce')



In [142]:
# Para nuestra variable objetivo, utilizaremos la columna 'PUNTAJE' que es la que representa la calidad del café.
# Esta columna contiene los puntajes de calidad asignados a cada lote de café

Y = df_full['PUNTAJE']
X = df_full.drop(columns=['LOTE', 'PUNTAJE', 'Lote'])

display(X.head())

print(X.dtypes)

,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,Arandanos,Azucar morena,Cacao,Caña de azucar,Cereza,Chocolate dulce,...,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste
0,Madre Laura,765.0,10.9,14,0,0,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,9.0,15.000000,7.65,Filtrado,175°/191°,08:01:00
1,Madre Laura,765.0,10.9,14,0,0,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,3.0,16.666667,2.50,Espressso,175°/195°,08:42:00
2,Madre Laura,765.0,10.9,14,0,0,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,9.0,16.444444,7.52,Filtrado,175°/190°,07:58:00
3,Madre Laura,765.0,10.9,14,0,0,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,11.7,16.324786,9.79,Filtrado,175°/191°,08:02:00
4,Madre Laura,765.0,10.9,14,0,0,0,0,0,0,...,Jerico,Dos mil,Tradicional,Lavado,29.0,15.344828,24.55,Filtrado,175°/192°,08:02:00


DENOMINACIÓN/     MARCA     object
CANTIDAD                   float64
%H                         float64
MALLAS                       int64
Arandanos                    int64
                            ...   
Merma                      float64
Peso en Tostado            float64
Perfil                      object
Temp. De inicio y final     object
Tiempo de tueste            object
Length: 110, dtype: object


In [143]:
# Ahora que separamos las variables independientes (X) de la dependiente (Y),
# procedemos a realizar el pipeline para procesar los datos cualitativos (categóricos).

Pipe = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

X_categorical = X.select_dtypes(include=['object'])

display(X_categorical.head())

# Procedemos a transformar las variables categóricas en variables numéricas (dummies) dentro del flujo de entrenamiento y validación de los modelos.
X_transformed = Pipe.fit_transform(X_categorical)
# Ahora unimos las variables transformadas con las variables numéricas
X_numerical = X.select_dtypes(exclude=['object'])
X_full = pd.concat([X_numerical, pd.DataFrame(X_transformed, columns=Pipe.get_feature_names_out())], axis=1)

# visualizamos las primeras filas del dataframe completo
X_full.head()

,DENOMINACIÓN/ MARCA,Origen,Variedad,Proceso,Beneficio,Perfil,Temp. De inicio y final,Tiempo de tueste
0,Madre Laura,Jerico,Dos mil,Tradicional,Lavado,Filtrado,175°/191°,08:01:00
1,Madre Laura,Jerico,Dos mil,Tradicional,Lavado,Espressso,175°/195°,08:42:00
2,Madre Laura,Jerico,Dos mil,Tradicional,Lavado,Filtrado,175°/190°,07:58:00
3,Madre Laura,Jerico,Dos mil,Tradicional,Lavado,Filtrado,175°/191°,08:02:00
4,Madre Laura,Jerico,Dos mil,Tradicional,Lavado,Filtrado,175°/192°,08:02:00


TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['str', 'time']